# Libraries

In [1]:
## Useful libraries
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
import copy
import pickle
from urllib.request import urlretrieve
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import TwoSlopeNorm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Additional input
import networkx as nx
from tqdm import tqdm
# !pip install torch_geometric
from torch_geometric.data import Data
# !pip install perlin-noise
from perlin_noise import PerlinNoise
import random
from loader import load_dataset
from datetime import datetime

from cycler import cycler
import seaborn as sns
import time

# Set the color scheme
sns.set_theme()
colors = ['#0076C2', '#EC6842', '#A50034', '#009B77', '#FFB81C', '#E03C31', '#6CC24A', '#EF60A3', '#0C2340', '#00B8C8', '#6F1D77']
plt.rcParams['axes.prop_cycle'] = cycler(color=colors)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from functions import *

# Dataset

In [2]:
data_folder = 'raw_datasets/'

train_dataset = 'DEM/'

In [3]:
dataset_folder = data_folder
n_sim = 80
start_sim = 1
dataset_name = 'grid'

datasets_folder = 'datasets'
if not os.path.exists(datasets_folder):
    os.makedirs(datasets_folder)
    
dataset_dir = datasets_folder + '/train'


##################### Use this code to create local pickle file #####################
pyg_dataset = create_grid_dataset(dataset_folder, n_sim=n_sim)
save_database(pyg_dataset, name=dataset_name, out_path=dataset_dir)

train_dataset = load_dataset(dataset_name=dataset_name, dataset_folder=dataset_dir)

100%|██████████| 80/80 [00:04<00:00, 16.55it/s]


In [4]:
def normalize_dataset(dataset, scaler_DEM, scaler_WD):
    
    min_DEM, max_DEM = scaler_DEM.data_min_[0], scaler_DEM.data_max_[0]
    min_WD, max_WD = scaler_WD.data_min_[0], scaler_WD.data_max_[0]
    normalized_dataset = []

    for idx in range(len(dataset)):
        DEM = dataset[idx]['DEM']
        WD = dataset[idx]['WD']
        norm_DEM = (DEM - min_DEM) / (max_DEM - min_DEM)
        norm_WD = (WD - min_WD) / (max_WD - min_WD)

        DEM = norm_DEM.reshape(64,64)
        WD = norm_WD[:,0].reshape(64,64)
        
        temp_dict = {}
        temp_dict['DEM'] = norm_DEM
        temp_dict['WD'] = norm_WD
        
        normalized_dataset.append(temp_dict)
        
        
        #temp_dict['Input'] = torch.stack((DEM, WD), dim=0)
        #WD_transposed = norm_WD[:, 1:].reshape(64,64, -1)
        #WD_transposed = WD_transposed.transpose(0, 2)
        #WD_transposed = WD_transposed.transpose(1, 2)
        #temp_dict['WD'] = WD_transposed
    
        #normalized_dataset.append(temp_dict)
    
    return normalized_dataset

In [5]:
# Normalize the inputs and outputs using training dataset
scaler_DEM = MinMaxScaler() # Can store DEM, VX, VY as one 'input' Scaler
scaler_WD = MinMaxScaler()

for index in range(len(train_dataset)): # =80
    scaler_DEM.partial_fit(train_dataset[index]['DEM'].reshape(-1, 1).cpu())
    scaler_WD.partial_fit(train_dataset[index]['WD'].reshape(-1, 1).cpu())
    

normalized_train_dataset = normalize_dataset(train_dataset, scaler_DEM, scaler_WD)
#normalized_test_dataset = normalize_dataset(test_dataset, scaler_DEM, scaler_WD)

In [6]:
# Split dataset into train, validation, and testing
train_percnt = 0.8
train_size = int(train_percnt * len(train_dataset))
val_size = len(train_dataset) - train_size
training_dataset, val_dataset = random_split(normalized_train_dataset, [train_size, val_size])

In [7]:
print(normalized_train_dataset[0].keys())

# Dataset has two variables, training data DEM, target WD
print('Amount of variables', len(normalized_train_dataset[0]))
print(f'Size of WD data ({len(normalized_train_dataset[0]["WD"])}, {len(normalized_train_dataset[0]["WD"][0])})')
print(normalized_train_dataset[0]['WD'])

dict_keys(['DEM', 'WD'])
Amount of variables 2
Size of WD data (4096, 97)
tensor([[0.0025, 0.0891, 0.0892,  ..., 0.1285, 0.1285, 0.1285],
        [0.0000, 0.0822, 0.0846,  ..., 0.2054, 0.2054, 0.2054],
        [0.0000, 0.0817, 0.1009,  ..., 0.3360, 0.3360, 0.3360],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


# Model

In [8]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()

        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias)]
        if batch_norm:
            layers.append(nn.BatchNorm2d(num_features=out_channels))
        layers.append(nn.PReLU())
        layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias))

        self.cnnblock = nn.Sequential(*layers)

    def forward(self, x):
        return self.cnnblock(x)

class Encoder(nn.Module):
    def __init__(self, channels=[32, 64, 128], kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()

        self.enc_blocks = nn.ModuleList([
            CNNBlock(channels[block], channels[block+1], kernel_size, padding, bias,
                     batch_norm=batch_norm)
            for block in range(len(channels)-1)]
            )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        outs = []
        for block in self.enc_blocks:
            x = block(x)
            outs.append(x)
            x = self.pool(x)
        return outs

class Decoder(nn.Module):
    def __init__(self, channels=[128, 64, 32], kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()
        self.channels = channels
        self.upconvs = nn.ModuleList([
            nn.ConvTranspose2d(channels[block], channels[block+1], kernel_size=2, padding=0, stride=2)
            for block in range(len(channels)-1)]
            )
        self.dec_blocks = nn.ModuleList([
            CNNBlock(channels[block], channels[block+1], kernel_size, padding, bias,
                     batch_norm=batch_norm)
             for block in range(len(channels)-1)]
             )

    def forward(self, x, x_skips):
        for i in range(len(x_skips)):
            x = self.upconvs[i](x)
            x = torch.cat((x, x_skips[-(1+i)]), dim=1)
            x = self.dec_blocks[i](x)

        x = self.dec_blocks[-1](x)
        return x

class CNN(nn.Module):
    def __init__(self, node_features, out_dim=1, n_downsamples=3, initial_hid_dim=64, batch_norm=True,
                 bias=True):
        super(CNN, self).__init__()
        hidden_channels = [initial_hid_dim*2**i for i in range(n_downsamples)]
        encoder_channels = [node_features]+hidden_channels
        decoder_channels = list(reversed(hidden_channels))+[out_dim]

        self.encoder = Encoder(encoder_channels, kernel_size=3, padding=1,
                               bias=bias, batch_norm=batch_norm)
        self.decoder = Decoder(decoder_channels, kernel_size=3, padding=1,
                               bias=bias, batch_norm=batch_norm)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x[-1], x[:-1])
        x = nn.Sigmoid()(x)
        return x

node_features = 2
model = CNN(node_features=node_features, n_downsamples=4, initial_hid_dim=32,
            batch_norm=True, bias=True)

In [9]:
def train_epoch(model, loader, optimizer, device='cpu'):
    model.to(device)
    model.train() # specifies that the model is in training mode

    losses = []

    for batch in loader:

        for timestep in range(normalized_train_dataset[0]['WD'].shape[1]-1):
            x = batch['DEM']   # [batch_size, channels, height, width]
            x = x.reshape(batch_size, 1, 64, 64)
            y0 = batch['WD'][:,:,timestep]
            y0 = y0.reshape(batch_size, 1, 64, 64)
            x = torch.cat((x,y0), 1)
            x = x.to(device)

            y = batch['WD'][:,:,timestep+1]
            y = y.reshape(batch_size, 1, 64, 64)
            y = y.to(device)

            # Model prediction
            preds = model(x)

            # MSE loss function
            loss = nn.MSELoss()(preds, y)

            losses.append(loss.cpu().detach())

            # Backpropagate and update weights
            loss.backward()   # compute the gradients using backpropagation
            optimizer.step()  # update the weights with the optimizer
            optimizer.zero_grad(set_to_none=True)   # reset the computed gradients

    losses = np.array(losses).mean()

    return losses

def evaluation(model, loader, batch_size, device='cpu'):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode

    losses = []

    with torch.no_grad():
        for batch in loader:

            x0 = batch['DEM']   # [batch_size, channels, height, width]
            x0 = x0.reshape(batch_size, 1, 64, 64)
            x0 = x0.to(device)
            y0 = batch['WD'][:,:,0]
            y0 = y0.reshape(batch_size, 1, 64, 64)
            y0 = y0.to(device)
            x = torch.cat((x0,y0), 1)

            # Model prediction

            timesteps = np.arange(0, normalized_train_dataset[0]['WD'].shape[1], 2)
            for timestep in timesteps:
            #for timestep in range(normalized_train_dataset[0]['WD'].shape[1]):
                y = batch['WD'][:,:,timestep]
                y = y.reshape(batch_size, 1, 64, 64)
                y = y.to(device)

                preds = model(x)

                # MSE loss function
                loss = nn.MSELoss()(preds, y)

                losses.append(loss.cpu().detach())

                x = torch.cat((x0,preds), 1)

    losses = np.array(losses).mean()

    return losses

In [10]:
# Set training parameters
learning_rate = 0.0005
batch_size = 16 ### 1, 2, 4, 5, 8, 10, 16, 20, 40, or 80
num_epochs = 10

# Create the optimizer to train the neural network via back-propagation
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Create the training and validation dataloaders to "feed" data to the model in batches
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#test_loader = DataLoader(normalized_test_dataset, batch_size=5, shuffle=False)

In [11]:
#create vectors for the training and validation loss
train_losses = []
val_losses = []

for epoch in range(1, num_epochs+1):
    # Model training
    train_loss = train_epoch(model, train_loader, optimizer, device=device)

    # Model validation
    val_loss = evaluation(model, val_loader, batch_size=batch_size, device=device)

    if epoch == 1:
        best_loss = val_loss

    if val_loss<=best_loss:
        best_model = copy.deepcopy(model)
        best_loss = val_loss
        best_epoch = epoch

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if epoch%10 == 0:
        print("epoch:",epoch, "\t training loss:", np.round(train_loss,4),
                            "\t validation loss:", np.round(val_loss,4))

model = copy.deepcopy(best_model)

epoch: 10 	 training loss: 1e-04 	 validation loss: 0.0064


In [12]:
#load_path = './models/model1.pth'
#model.load_state_dict(torch.load(load_path, map_location=torch.device(device)))

In [13]:
test_loss = evaluation(model, test_loader, batch_size=5, device=device)
print(test_loss)

NameError: name 'test_loader' is not defined

In [ ]:
plt.plot(train_losses, label='Training')
plt.plot(val_losses, label='Validation')
plt.yscale('log')
plt.title('Losses')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
# select one sample
data_id = 60
timestep = normalized_train_dataset[0]['WD'].shape[1] - 2   # can only be even numbers for comparing to predictions
timestep_pred = int(timestep/2)

x = normalized_train_dataset[data_id]['DEM'].reshape(1, 1, 64, 64)
WD = normalized_train_dataset[data_id]['WD'][:,timestep+1]

x = x.to(device)

show_x = x.reshape(64,64).cpu()
show_WD = WD.reshape(64,64).cpu()

timestep0 = normalized_train_dataset[data_id]['WD'][:,0].reshape(1, 1, 64, 64).to(device)
input = torch.cat((x, timestep0), 1)
for i in range(int(normalized_train_dataset[0]['WD'].shape[1]/2)):
#for i in range(normalized_train_dataset[0]['WD'].shape[1]):
    y = model(input)
    if i == 0:
        pred_WD = y.detach()
    else:
        pred_WD_new = y.detach()
        pred_WD = torch.cat((pred_WD, pred_WD_new), 1)
    input = torch.cat((x, y), 1)

show_pred_WD = pred_WD[0, timestep_pred, :, :].cpu()

In [ ]:
plt.imshow(show_x, cmap='terrain', origin='lower')

In [ ]:
plt.imshow(show_WD, cmap='terrain', origin='lower')

In [ ]:
plt.imshow(show_pred_WD, cmap='terrain', origin='lower')

In [ ]:
DEM = scaler_DEM.inverse_transform(x[0].reshape(1,-1).T.cpu())[:,0].reshape(64,64)
real_WD = scaler_WD.inverse_transform(WD.reshape(-1,1).cpu()).reshape(1,64,64)[0]
pred_WD = scaler_WD.inverse_transform(pred_WD[0, timestep_pred, :, :].reshape(-1,1).cpu()).reshape(64,64)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(17,5))

diff_WD = real_WD - pred_WD
max_WD = max(pred_WD.max(), real_WD.max())
max_diff = max(diff_WD.max(), -diff_WD.min())

axs[0].imshow(DEM.squeeze(), cmap='terrain', origin='lower')
axs[1].imshow(real_WD.squeeze(), vmin = 0, vmax=max_WD, cmap='Blues_r', origin='lower')
axs[2].imshow(pred_WD.squeeze(), vmin = 0, vmax=max_WD,cmap='Blues_r', origin='lower')
axs[3].imshow(diff_WD.squeeze(), vmin=-max_diff, vmax=max_diff, cmap='RdBu', origin='lower')

plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = DEM.min(), vmax=DEM.max()),
                            cmap='terrain'), fraction=0.05, shrink=0.9, ax=axs[0])
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = 0, vmax=max_WD),
                            cmap='Blues_r'), fraction=0.05, shrink=0.9, ax=axs[1])
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = 0, vmax=max_WD),
                            cmap='Blues_r'), fraction=0.05, shrink=0.9, ax=axs[2])
plt.colorbar(plt.cm.ScalarMappable(norm=TwoSlopeNorm(vmin=-max_diff, vmax=max_diff, vcenter=0),
                            cmap='RdBu'), fraction=0.05, shrink=0.9, ax=axs[3])
for ax in axs:
    ax.axis('off')

axs[0].set_title('DEM')
axs[1].set_title('Real WD (h)')
axs[2].set_title('Predicted WD (h)')
axs[3].set_title('Difference (h)')

plt.show()